# Flowlines Extraction
Work in progress. Extract velocity data at flowlines to construct time series of them.

---

## 1. Load packages and open datasets

In [2]:
from IPython.lib.deepreload import reload
%load_ext autoreload
%autoreload 2
import nisardev as nisar
import os
import matplotlib.colors as mcolors
import grimpfunc as grimp
import matplotlib.pyplot as plt
import geopandas as gpd
from datetime import datetime
import numpy as np
import xarray as xr
import importlib
import dask
from dask.diagnostics import ProgressBar
ProgressBar().register()
dask.config.set(num_workers=2)  # Avoid problems with too many open connections at NSIDC
import glob
import panel
panel.extension()

In [3]:
ziVelSeries = nisar.nisarVelSeries()
ziVelSeries.readSeriesFromNetCDF('./data/raw/ZachariaeIsstromVelocitySeries.nc')
ziFlowlines = gpd.read_file('./data/raw/Flowlines_Felikson_shp/glacier0115.shp')

### 1.1 Print data for visual analysis

In [9]:
ziFlowlines

,flowline,geometry
0,03,"LINESTRING (497822.495 -1100444.834, 497572.49..."
1,04,"LINESTRING (498154.637 -1098438.063, 497904.63..."
2,05,"LINESTRING (496710.341 -1096994.329, 496460.34..."
3,06,"LINESTRING (496475.023 -1095140.529, 496225.02..."
4,07,"LINESTRING (495177.218 -1093630.355, 494927.21..."
5,08,"LINESTRING (492086.096 -1093189.529, 491836.09..."


In [21]:
ziVelSeries.xr

<xarray.DataArray 'VelocitySeries' (time: 443, band: 3, y: 301, x: 301)> Size: 482MB
[120408729 values with dtype=float32]
Coordinates:
  * time         (time) datetime64[ns] 4kB 2015-01-06T12:00:00 ... 2024-04-18...
    id           (time) <U115 204kB ...
  * band         (band) <U2 24B 'vx' 'vy' 'vv'
  * x            (x) float64 2kB 4.4e+05 4.402e+05 4.404e+05 ... 4.998e+05 5e+05
  * y            (y) float64 2kB -1.08e+06 -1.08e+06 ... -1.14e+06 -1.14e+06
    epsg         int64 8B ...
    name         <U4 16B ...
    time1        (time) datetime64[ns] 4kB 2015-01-01 2015-01-13 ... 2024-04-13
    time2        (time) datetime64[ns] 4kB 2015-01-12 2015-01-24 ... 2024-04-24
    _FillValue   float64 8B ...
    spatial_ref  int64 8B ...
Attributes:
    grid_mapping:  spatial_ref

## 2. Convert linestrings to arrays of x and y points

In [15]:
def linestring_to_points(feature, line):
    x_coords, y_coords = zip(*line.coords)
    return list(x_coords), list(y_coords)

In [16]:
ziFlowlines['points'] = ziFlowlines.apply(lambda l: linestring_to_points(l['flowline'], l['geometry']), axis=1)

In [22]:
velMean = ziVelSeries.mean()

In [10]:
fig, axes = plt.subplots(1, 2, figsize=(14,7))
mapAxes, plotAxes = axes
# Map plot
velMean.displayVelForDate(ax=mapAxes, units='km', autoScale=True, axisOff=True, scale='log',
                                 midDate=False, colorBarLabel='Mean Speed (m/yr)', 
                                 extend='both', backgroundColor=(0.6, 0.6, 0.6), colorBarPosition='left', vmin=1, colorBarSize='3%', colorBarPad=.1)
# Loop to plot points
for x, y, symbol, color, label in zip(x_coords, y_coords, colors):
    # plot points on map
    mapAxes.plot(x, y, symbol, color=color, markersize=10) 
    # plot values only if they are not 0 or null
    ziVelSeries.plotPoint(x, y, ax=plotAxes, band='vv',  marker=symbol, color=color,  linestyle='-',
                        units='km', label=label, markersize=9)
# Finish plots
ziVelSeries.labelPointPlot(xLabel='Year', yLabel='Speed (m/yr)', ax=plotAxes, title='Speeds for Zachariæ Isstrøm')
plotAxes.legend(fontsize=14)

AttributeError: 'nisarVelSeries' object has no attribute 'getVelocityAtPoint'